In [1]:
import pandas as pd
import numpy as np

In [ ]:
from snowflake.cortex import complete
from trulens.core.otel.instrument import instrument
from trulens.otel.semconv.trace import SpanAttributes

class Multi_Agent_App:

    def agent_1_cortex_analyst(self, client: str) -> list:
        """
        takes client and inserts into predefined prompt and passes to cortex analyst to return list of products client uses today
        """
        request_body = {
            "messages": [f"Show me all products used by {client} today"],
            "semantic_model_file": f"@{DATABASE}.{SCHEMA}.{STAGE}/{FILE}",
        }
        resp = _snowflake.send_snow_api_request(...)

        products = session.sql(resp).collect()

        return products
    
    def agent_2_web_serach(self, client: str) -> list:
        """ 
        calls api to google client name and return top three new articles. Use bs4 or similar library to parse web results
        take parsed web result and pass to cortex LLM for summariziation
        """

        

    return cortex_llm_summary


    def agent_3_web_serach(self, client: str) -> list:
        """ 
        calls api to google client name and return top three articles. Use bs4 or similar library to parse web results
        take parsed web result and pass to cortex LLM for summariziation
        """
    return cortex_llm_summary

    def agent_4_prep_pitch(self, client: str) -> list:
        """ 
        calls api to google client name and return top three articles. Use bs4 or similar library to parse web results
        take parsed web result and pass to cortex LLM for summariziation
        """
        products = self.agent_1_cortex_analyst(client)
        news = self.agent_2_web_search(client)
        news = self.agent_2_web_search(client)
        prep_pitch = complete('claude-3-5-sonnet', f"prepare a product pitch based on the following products being used and the{products}")
    return cortex_llm_summary




rag = RAG()